In [2]:

import os
import requests
import json
import openai
# from langchain.llms import AzureOpenAI
import openai
import time
from extract import *
from datetime import datetime
now = datetime.now()
import warnings
warnings.filterwarnings("ignore")
from dotenv import load_dotenv
from utilities import *
from configs import *
import uuid
load_dotenv()

# initializing application variables
openai.api_key = os.environ.get("AZURE_OPENAI_KEY")
openai.api_base = os.environ.get("AZURE_OPENAI_ENDPOINT") 
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future
deployment_name=os.environ.get('DEPLOYMENT_NAME')

#read from prompt json file in blob
prompt=read_from_blob(prompt_json_folder,prompt_json_filename)
prompt = json.loads(prompt.decode('utf-8'))
prompt_ocr=prompt['prompt 1']['prompt']
# prompt_xml=prompt['prompt 2']['prompt']

#read spl key from blob
spl_key=read_from_blob(spl_key_folder,spl_key)
spl_key_df=pd.read_csv(BytesIO(spl_key))
ndc_list=list(spl_key_df['RawNDC'].value_counts().index)

#read blob images from batch folder
blob_list=read_blob_batch(batch_folder_name)

response={'NDC':'','Storage_Condition':'','Reason':'','Status':'','created_date':'','GUID':'','execution_time':'','prompt_id':'','error_msg':'','response_status_code':'','batch_id':'','img_loc':''}
results=pd.DataFrame(columns=response.keys())
count=0

for blob in blob_list:
    print(blob.name)
    # count+=1
    # if count==10:
    #     break
    
    response={'NDC':'','Storage_Condition':'','Reason':'','Status':'','created_date':'','GUID':'','execution_time':'','prompt_id':'','error_msg':'','response_status_code':'','batch_id':'','img_loc':''}
    
    if ".jpg" in blob.name or ".png" in blob.name:
        # print(blob.name)
        t1=time.time()
        img_bytes=read_from_blob('',blob.name)
        ocr_text=ocr(img_bytes)
        ocr_text=extract_text(ocr_text[0])
        # print(ocr_text)
        if ocr_text==None:
            #logging
            continue
                
        GUID=str(uuid.uuid4())
        response['GUID']=GUID
        response['img_loc']=str(blob.name)
        
        #upload ocr to blob
        upload_json_to_blob(ocr_folder,GUID,ocr_text)
        
        ndc=ndc_conversion(ndc_regex(ocr_text))
        print(ndc)
        if ndc==None:
            #logging
            continue
        
        elif ndc !=None and ndc in ndc_list:
            #response from openai
            gpt_response=llm_response(ocr_text,prompt_ocr,ndc)
            # print(gpt_response)
            response['Storage_Condition']=gpt_response['Storage_Condition']
            response['Reason']=gpt_response['Reason']
            response['created_date']=now.strftime("%m/%d/%Y %H:%M:%S")
            response['prompt']='prompt 1'               
            if response_validation(response):
                response['Status']='success'
            else:
                response['Status']='failure'
                response['Reason']='Response Validation failed'
                
        elif ndc !=None and ndc not in ndc_list:
            response['Status']='failure'
            response['Reason']= 'NDC not in spl key'
            response['NDC']=ndc
            response['created_date']=now.strftime("%m/%d/%Y %H:%M:%S")
            #logging
            continue
        
        t2 = time.time()
        execution_time=t2-t1
        response['execution_time']=execution_time
        
        for i in response.keys():
            response[i]=[response[i]]
            
        results=pd.concat([results,pd.DataFrame(response)],axis=0)
        
    else:
        continue
print('results',results)            
spl_key_df.rename(columns = {"RawNDC": "NDC","NDC":"NDC10"},inplace = True)
results=results.merge(spl_key_df[['ProprietaryName','DocID','SetID','NDC','S3Key','NDC10','NDC11','S3Key']],on='NDC',how='left')
print('results after merge',results)

#fetching unique ndcs
plan_a=unique_ndc(results)
print('plan A df',plan_a)

dff=plan_a[plan_a['Storage_Condition']=='Undetermined']

print('dff',dff)
#implement plan b which will fetch responses for dff and then update in plan a and create a final_df


# upload_to_table(response,final_df)


Druglabelinput/2022
Druglabelinput/2022/02
Druglabelinput/2022/02/03
Druglabelinput/2022/02/03/ 29df8839-3dfa-0b2b-c5a3-906db6997b3b
Druglabelinput/2022/02/03/ 29df8839-3dfa-0b2b-c5a3-906db6997b3b/Screen Shot 2014-09-23 at 1.10.15 PM.jpg
50268-728-15
Druglabelinput/2022/02/03/ 29df8839-3dfa-0b2b-c5a3-906db6997b3b/Screen Shot 2014-11-04 at 3.18.19 PM.jpg
50268-729-13
Druglabelinput/2022/02/03/ 29df8839-3dfa-0b2b-c5a3-906db6997b3b/a8f9a591-05db-4493-9296-630e9c8c2a70.xml
Druglabelinput/2022/02/03/ 29df8839-3dfa-0b2b-c5a3-906db6997b3b/sulfatrime-str1.jpg
None
Druglabelinput/2022/02/03/ 29df8839-3dfa-0b2b-c5a3-906db6997b3b/sulfatrime-str2.jpg
None
Druglabelinput/2022/02/14
Druglabelinput/2022/02/14/abc1d07f-91ee-4421-bc9b-457d0ed41f4c
Druglabelinput/2022/02/14/abc1d07f-91ee-4421-bc9b-457d0ed41f4c/0db13470-fcc5-4378-86ab-20f36277ab3c.xml
Druglabelinput/2022/02/14/abc1d07f-91ee-4421-bc9b-457d0ed41f4c/image-01.jpg
None
Druglabelinput/2022/02/14/abc1d07f-91ee-4421-bc9b-457d0ed41f4c/image-02.jp

In [ ]:
#read from prompt json file in blob
prompt=read_from_blob(prompt_json_folder,prompt_json_filename)
prompt = json.loads(prompt.decode('utf-8'))
prompt_ocr=prompt['prompt 1']['prompt']

In [1]:
import json
prompt = json.load(open('prompt.json','r'))
# prompt_ocr=prompt['prompt 1']['prompt']

In [6]:
print(prompt[0]['prompt'])


You will be receiving the OCR text for a drug label image in backticks.
Your first step is to extract the NDC out of the OCR text. NDC is in the following format in single quotes:
'0002-0800-01'. You have to extract the drug storage condition from the given text. Make sure you understand 
that we need the details prior to reconstitution and not after the sale, from a Pharmacist's perspective. 
You need to classify the storage condition in the following categories given below along with their 
descriptions and conditions:

1.Room Temperature
Product can be stored at room temperature prior to sale/dispensing
Usually temperature between 15 degree -25 degree C (59 degree -77 degree F)

2.Refrigerate
Product should be refrigerated prior to sale/dispensing
Usually temperature between 2 degree -8 degree C (36 degree -46 degree F)

3.Freeze
Product should be kept frozen prior to sale/dispensing
At or below 0 degree C (32 degree F) temperature.

4.Undetermined
When no NDC is provided or when no